# Preambule
#### Goal
The goal of this session is to get familiar with the Bloomberg Python API.<br> 
This will be done by building a class containing a function which mimicks the behavior of the BDH Excel function.

#### What the function will do
Our BDH-like function should be able to : <br>
1 - Retrieve historical data <br>
2 - For as many tickers as possible <br>
3 - For as many fields as possible <br>
4 - Include the various options <br>
5 - And allow for the possibility to add overrides <br>

#### References 
https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

# I. Dependencies
These are the libraries we will be using in this notebook. blpapi is the library used for Bloomberg data.

In [1]:
# pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple/ blpapi
import blpapi
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

# II. Set up the Bloomberg names

We here create variables using the Name class within blpapi. <br> 
This will allow to write cleaner and more concise code when refering to strings with the api.<br>
Below are only the names required for our present work. Many more exist and you can refer to the different examples within the SDK for ones of interest to your task.

In [2]:
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

# III. The BLP class
We now start to build our function within a dedicated class.<br>

A brief reminder on the class object in Python:<br>
- Classes must have a function called _\_init_\_() which is automatically executed at class initiation
- Classes can have one or several methods
- Class object need to be instaciated before using its methods

#### A. The init function

This function aims at starting the session and setting up the desired service 

#### B. The close session method:
Simply kills the session so no ghost connection remains. 

#### C. The BDP method:
3 steps: <br>
1- Create request<br>
2- Send request <br>
3- Extract data<br>




In [3]:
class BLP():
    #-----------------------------------------------------------------------------------------------------    
   
    def __init__(self):
        """
            Improve this
            BLP object initialization
            Synchronus event handling
           
        """
        # Create Session object
        self.session = blpapi.Session()
       
       
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
       
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
       
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')
 
        #print('Session open')
   
    #-----------------------------------------------------------------------------------------------------
   
    def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', currency = ""):
        """
            Summary:
                HistoricalDataRequest ;
       
                Gets historical data for a set of securities and fields
 
            Inputs:
                strSecurity: list of str : list of tickers
                strFields: list of str : list of fields, must be static fields (e.g. px_last instead of last_price)
                startdate: date
                enddate
                per: periodicitySelection; daily, monthly, quarterly, semiannually or annually
                perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
                curr: string, else default currency is used
                Days: nonTradingDayFillOption : NON_TRADING_WEEKDAYS*, ALL_CALENDAR_DAYS or ACTIVE_DAYS_ONLY
                fill: nonTradingDayFillMethod :  PREVIOUS_VALUE, NIL_VALUE
               
                Options can be selected these are outlined in “Reference Services and Schemas Guide.”    
           
            Output:
                A list containing as many dataframes as requested fields
            # Partial response : 6
            # Response : 5
           
        """
           
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
       
        # Create request
        request = self.refDataSvc.createRequest('HistoricalDataRequest')
       
        # Put field and securities in list is single value is passed
        if type(strFields) == str:
            strFields = [strFields]
           
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
   
        # Append list of securities
        for strF in strFields:
            request.append('fields', strF)
   
        for strS in strSecurity:
            request.append('securities', strS)
   
        # Set other parameters
        request.set('startDate', startdate.strftime('%Y%m%d'))
        request.set('endDate', enddate.strftime('%Y%m%d'))
        request.set('periodicitySelection', per)
        request.set('periodicityAdjustment', perAdj)
        request.set('nonTradingDayFillMethod', fill)
        request.set('nonTradingDayFillOption', days)
        if(currency!=""):  
            request.set('currency', currency)
 
        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------
 
        requestID = self.session.sendRequest(request)
        print("Sending request")
       
        #-----------------------------------------------------------------------
        # Receive request
        #-----------------------------------------------------------------------
       
        dict_Security_Fields={}
        liste_msg = []
        while True:
            event = self.session.nextEvent()
           
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
           
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            liste_msg.append(msg)
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break
   
        #-----------------------------------------------------------------------
        # Exploit data
        #----------------------------------------------------------------------
       
        # Create dictionnary per field
        dict_output = {}
        for field in strFields:
            dict_output[field] = {}
            for ticker in strSecurity:
                dict_output[field][ticker] = {}
                 
        # Loop on all messages
        for msg in liste_msg:
            countElement = 0
            security_data = msg.getElement(SECURITY_DATA)
            security = security_data.getElement(SECURITY).getValue() #Ticker
            # Loop on dates
            for field_data in security_data.getElement(FIELD_DATA):
               
                # Loop on differents fields
                date = field_data.getElement(0).getValue()
               
                for i in range(1,field_data.numElements()):
                    field = field_data.getElement(i)
                    dict_output[str(field.name())][security][date] = field.getValue()
                   
                countElement = countElement+1 if field_data.numElements()>1 else countElement
                
            # remove ticker
            if countElement==0:
                for field in strFields:
                    del dict_output[field][security]
                   
        for field in dict_output:
            dict_output[field] = pd.DataFrame.from_dict(dict_output[field])
        return dict_output  

    #-----------------------------------------------------------------------------------------------------

    def bdp(self, strSecurity, strFields, strOverrideField='', strOverrideValue=''):
        
        """
            Summary:
                Reference Data Request ; Real-time if entitled, else delayed values 
                Only supports 1 override
                
                
            Input:
                strSecurity
                strFields
                strOverrideField
                strOverrideValue         
            
            Output:
               Dict 
        """
        
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request
        request = self.refDataSvc.createRequest('ReferenceDataRequest')
        
        # Put field and securities in list is single field passed
        if type(strFields) == str:
            strFields = [strFields]
        
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
            
        # Append list of fields
        for strD in strFields:
            request.append('fields', strD)

        # Append list of securities
        for strS in strSecurity:
            request.append('securities', strS)

        # Add override 
        if strOverrideField != '':
            o = request.getElement('overrides').appendElement()
            o.setElement('fieldId', strOverrideField)
            o.setElement('value', strOverrideValue)

        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------

        requestID = self.session.sendRequest(request)
        # print("Sending request")

        #-----------------------------------------------------------------------
        # Receive request                
        #-----------------------------------------------------------------------
                
        list_msg = []
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            list_msg.append(msg)
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break    

        #-----------------------------------------------------------------------
        # Extract the data 
        #-----------------------------------------------------------------------
        dict_output = {}        
        for msg in list_msg:

            for security_data in msg.getElement(SECURITY_DATA):
                ticker = security_data.getElement(SECURITY).getValue() #Ticker
                dict_output[ticker] = {}
                for i in range(0, security_data.getElement(FIELD_DATA).numElements()): # on boucle sur les fields
                    fieldData = security_data.getElement(FIELD_DATA).getElement(i)
                    dict_output[ticker][str(fieldData.name())] = fieldData.getValue()


        return pd.DataFrame.from_dict(dict_output).T
    
    def bds(self, strSecurity, strFields, strOverrideField='', strOverrideValue=''):
        
        """
            Summary:
                Reference Data Request ; Real-time if entitled, else delayed values 
                Only supports 1 override
                
                
            Input:
                strSecurity
                strFields
                strOverrideField
                strOverrideValue         
            
            Output:
               Dict 
        """
        
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request
        request = self.refDataSvc.createRequest('ReferenceDataRequest')
        
        # Put field and securities in list is single field passed
        if type(strFields) == str:
            strFields = [strFields]
        
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
            
        # Append list of fields
        for strD in strFields:
            request.append('fields', strD)

        # Append list of securities
        for strS in strSecurity:
            request.append('securities', strS)

        # Add override 
        if strOverrideField != '':
            o = request.getElement('overrides').appendElement()
            o.setElement('fieldId', strOverrideField)
            o.setElement('value', strOverrideValue)

        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------

        requestID = self.session.sendRequest(request)
        #print("Sending request")

        #-----------------------------------------------------------------------
        # Receive request                
        #-----------------------------------------------------------------------
                
        list_msg = []
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            list_msg.append(msg)
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break    
        
        #-----------------------------------------------------------------------
        # Extract the data 
        #-----------------------------------------------------------------------
        dict_output = {}
        for msg in list_msg:
            for security_data in msg.getElement(SECURITY_DATA): #Boucle sur les tickers
                ticker = security_data.getElement(SECURITY).getValue() #Ticker
                dict_field = []
                for field_data in security_data.getElement(FIELD_DATA).getElement(0): #Boucle sur les données des fields
                    
                    dict_elements = {}
                    for i in range(field_data.numElements()): 
                        dict_elements[str(field_data.getElement(i).name())] = str(field_data.getElement(i).getValue())
                    dict_field.append(dict_elements)
                
                dict_output[ticker] = pd.DataFrame(dict_field)
        
        return dict_output
    
    #-----------------------------------------------------------------------------------------------------
 
    def closeSession(self):    
        #print("Session closed")
        self.session.stop()

## IV. Loading of the necessary Data

### Récupération de la composition historique de l'indice à chaque date

In [ ]:
index = ['SPX Index']

In [ ]:
# Créer une liste de dates entre janvier 2000 et aujourd'hui avec une fréquence mensuelle
start_date = datetime(2002, 1, 15)
end_date = datetime.today()
dates = pd.date_range(start=start_date, end=end_date, freq='M')

# Convertir les dates au format Ymd collé
dates_str = dates.strftime('%Y%m%d')
strFields = ["INDX_MWEIGHT_HIST"]

In [ ]:
# Initialisation de la session BLP
df_fin = pd.DataFrame(index=dates_str)  # DataFrame avec les dates comme index
blp = BLP()  
for i in range(len(dates_str)):
    
    # Récupérer les données de Bloomberg pour chaque date
    df = blp.bds(strSecurity=index, strFields=strFields, strOverrideField="END_DATE_OVERRIDE", strOverrideValue=dates_str[i])
    # Récupérer les tickers uniques pour la date donnée
    tickers = list(df['SPX Index']['Index Member'].unique())
    # Ajouter les colonnes pour les tickers si elles n'existent pas encore
    for ticker in tickers:
        if ticker not in df_fin.columns:
            df_fin[ticker] = 0  # Ajouter une colonne pour ce ticker
            
    # Mettre 1 pour chaque ticker présent à la date donnée
    df_fin.loc[dates_str[i], tickers] = 1
    # Fermer la session Bloomberg après le traitement
blp.closeSession()

# Afficher le DataFrame final
print(df_fin)


C:\Users\dufoba24\AppData\Local\Temp\ipykernel_11368\998565136.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fin[ticker] = 0  # Ajouter une colonne pour ce ticker
C:\Users\dufoba24\AppData\Local\Temp\ipykernel_11368\998565136.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fin[ticker] = 0  # Ajouter une colonne pour ce ticker
C:\Users\dufoba24\AppData\Local\Temp\ipykernel_11368\998565136.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

          0111145D UN  0202445Q UN  0203524D UN  0226226D UN  0544749D UN  \
20020131            1            1            1            1            1   
20020228            1            1            1            1            1   
20020331            1            1            1            1            1   
20020430            1            1            1            1            1   
20020531            1            1            1            1            1   
...               ...          ...          ...          ...          ...   
20240930            0            0            0            0            0   
20241031            0            0            0            0            0   
20241130            0            0            0            0            0   
20241231            0            0            0            0            0   
20250131            0            0            0            0            0   

          0772031D UN  0848680D UN  0910150D US  0948669D UN  0961514D UN  

C:\Users\dufoba24\AppData\Local\Temp\ipykernel_11368\998565136.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fin[ticker] = 0  # Ajouter une colonne pour ce ticker
C:\Users\dufoba24\AppData\Local\Temp\ipykernel_11368\998565136.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fin[ticker] = 0  # Ajouter une colonne pour ce ticker


In [ ]:
# Sauvegarder le DataFrame final dans un fichier Excel
df_fin.to_excel("COMPO_SP500.xlsx")

### Récupération des facteurs

In [ ]:
df = pd.read_excel("COMPO_SP500.xlsx")

In [ ]:
tickers = list(df.iloc[:,1:].columns)
# Renormalisation du nom des tickers
tickers = [tick + " Equity" for tick in tickers]

In [ ]:
fields = ['RETURN_COM_EQY', 'PX_TO_BOOK_RATIO', 'PX_LAST']
 
start_date = dt.datetime(2002, 1, 15)
end_date = dt.datetime(2025, 1, 31)
 
 
blp = BLP()
 
dict_bdh = blp.bdh(strSecurity = tickers, strFields = fields, startdate = start_date, enddate = end_date, per = 'DAILY')
print(dict_bdh)
for field, df_field in dict_bdh.items():
    df_field.to_excel(f"{field}.xlsx", index=True)

['0111145D UN Equity', '0202445Q UN Equity', '0203524D UN Equity', '0226226D UN Equity', '0544749D UN Equity', '0772031D UN Equity', '0848680D UN Equity', '0910150D US Equity', '0948669D UN Equity', '0961514D UN Equity', '0964591D UQ Equity', '1028411Q UN Equity', '1040983D UQ Equity', '1086832D UN Equity', '1255173D UN Equity', '1280712D UQ Equity', '1281683D UN Equity', '1284849D UN Equity', '1288652D US Equity', '1293405D UN Equity', '1312089D UN Equity', '1317355D UN Equity', '1431816D UN Equity', '1436513D UN Equity', '1500785D UN Equity', '1518855D US Equity', '1519128D UQ Equity', '1528159D UN Equity', '1541931D UQ Equity', '1649858D UQ Equity', '1683351D UQ Equity', '1683997D UQ Equity', '1684442D UN Equity', '1697067D UN Equity', '1704453D UN Equity', '1715651D UN Equity', '1719868D UN Equity', '1746513D UQ Equity', '1752754D UN Equity', '1799685D UN Equity', '1822785D UN Equity', '1825471D UN Equity', '1831877D UN Equity', '1837572D UN Equity', '1858968D UN Equity', '1920486D